# 计算思维数据处理

数据处理要求：
### 1、每个题的平均作答时长；——结果数据
### 2、每个题的编码种类（有多少种，分别是什么，每种多少学生）；——结果数据
3、每个题的每个操作步骤的平均作答时长；——过程数据
### 4、正确率（暂未提供标准编码，可以探索一下题目本身，协助形成标准答案编码）；——结果数据
5、关键节点（通过数据，探索学生在从初始状态向终止状态进行的过程中，有几个关键步骤，每个关键步骤有几种类型的关键节点编码），体现“用数据说话”去探索关键节点。——过程数据
6、每个题目的每种编码下，都有什么样的学生作答类型，比如都是正确的，但是可以聚成多少类，每一类有什么特征，学生是通过什么样的操作路径到达最终的。
### 7、每道题目的正确率；——结果数据

<font color="red">注意 要求点四和要求点七相同，将一同分析

## 0、数据加载和预处理
首先读取数据，并通过pandas进行数据帧处理
为了排版整洁和处理方便，以及保证工具的可拓展性，这里将处理工具封装成`data_analysis`的python类，并在jupyter中调用，该类在jupyter notebook同文件夹下的`main.py`文件中

In [34]:
# 首先引入需要的第三方库
import pandas as pd
import json 
import numpy as np
import ast
from datetime import datetime
import plotly.graph_objs as go
from plotly.offline import plot
import plotly.offline as offline
from pandas.core.indexes import interval
import plotly.figure_factory as ff
pyolt=plot
import plotly.express as px
import math
import re

In [2]:
# 从main.py中引用对应的工具
from main import data_analysis

In [3]:
# 首先将excel文件读取为pandas的dataframe类型
# 然后将该dataframe作为参数以初始化对应的数据处理工具，这里没有对excel文件进行预处理，
#   超时数据还未删除，并且并没有对不同学校进行学生分类，所以命名为'df_all'
df_all = pd.read_excel('./data/ticket_user_mianyang.xlsx')  
# dataframe并不直接对其进行处理，而是作为参数初始化一个类的实体，这样的好处是可以避免大量的代码冗余
#    在对不同学校和不同数据行进行分类后处理时，只需要额外生成新的类的实体即可
#    在这里对应所有数据行直接生成一个实体，命名为'df_all_entity'
#    注意在参数里有一个命名为'name'的参数，这里是方便在调试过程中快速判断出问题的是哪个dataframe
df_all_entity =  data_analysis(df = df_all, name = 'all')
# 该步骤运行时间较长，在32秒左右

init complete


### 初始化完成
初始化完成的实体中有以下几个属性：
self.df
    这是未经任何处理的原始表格数据
self.count_df_list
    这是初步对答案编码进行归类并计数的dataframe的列表，该列表中包含22个dataframe，其中每个dataframe里针对不同回答的编码进行分类并计数。注意，这里只是初步计数，有某些编码再更抽象的意义上是等效的，这一点会在之后统计正确率的时候统一处理

In [4]:
print("题目个数为：",len(df_all_entity.count_df_list))
print("展示22个datafame中的第0个：")
df_all_entity.count_df_list[0]

题目个数为： 22
展示22个datafame中的第0个：


,0
ans_str,
None,1
"[0, 180, 'SmoothScroll']",1
"[0, 2, 3, [0, 1, 3, 2]]",46
"[0, 2, 3, [0, 2, 3, 4]]",91
"[0, 2, 3, [0, 2, 4, 3]]",87
"[0, 2, 3, [0, 3, 1, 2]]",121
"[0, 2, 3, [0, 3, 2, 1]]",62
"[0, 2, 3, [0, 4, 2, 1]]",47
"[0, 2, 3, [1, 2, 3, 4]]",1082


## 每道题的编码种类和正确率分析

In [35]:
accuracy_list = []
addition_list = []
for i, df in enumerate(df_all_entity.count_df_list):
    print(i)
    additional_infor_df = pd.DataFrame({'list':[ast.literal_eval(index) for index in df.index]})
    additional_infor_df.insert(len(additional_infor_df.columns), 'count', list( df.iloc[:, 0]))
    additional_infor_df.to_excel('./output/all'+'/' +str(i) + '_count.xlsx')
    if i in [0, 1]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) != 4:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)
        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['0' if l==None else str(l[0]) for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [2,3]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) == 0:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != str:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)

        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)!=0 and l[0]=='00' else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)  
    elif i in [5]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) == 0:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != str:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)
        
        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==2 and l[0]+l[1]=='B_AC_A' else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [6]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) == 0:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != str:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)

        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==5 and l[0]+l[1]+l[2]+l[3]+l[4]=='B_AC_AG_FD_BE_B' else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [7]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) != 4:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != list or type(additional_infor_df.iloc[j,0][1]) != list or type(additional_infor_df.iloc[j,0][2]) != list or type(additional_infor_df.iloc[j,0][3]) != list:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)

        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==4 and [[0,1],[0,2],[1,2]] in l and [[0,4],[0,5],[1,5]] in l and [[0,6],[0,7],[1,7]] in l and [[0,10],[0,11],[1,11]] else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [8] :
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) != 5:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != list or type(additional_infor_df.iloc[j,0][1]) != list or type(additional_infor_df.iloc[j,0][2]) != list or type(additional_infor_df.iloc[j,0][3]) != list or type(additional_infor_df.iloc[j,0][4]) != list:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)

        verify_list = [[[1, 0], [2, 0], [3, 0], [3, 1]], [[0, 1], [1, 1], [2, 1], [2, 2]], [[3, 2], [4, 2], [5, 2], [5, 3]], [[2, 3], [3, 3], [4, 3], [4, 4]], [[0, 4], [1, 4], [2, 4], [2, 5]]]
        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==5 and verify_list[0] in l  and verify_list[1] in l  and verify_list[2] in l  and verify_list[3] in l  and verify_list[4] in l else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [9]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) == 0:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != str:
                drop_index_list.append(j)
            
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)
        for row in range(len(additional_infor_df)):
            list_temp = additional_infor_df.loc[row,'list']
            if list_temp!=None and len(list_temp)==2:
                if list_temp[0][0:2] > list_temp[0][-2:]:
                    list_temp[0] = list_temp[0][-2:] + '_' + list_temp[0][0:2]
                if list_temp[1][0:2] > list_temp[1][-2:]:
                    list_temp[1] = list_temp[1][-2:] + '_' + list_temp[1][0:2]
                if list_temp[0] > list_temp[1]:
                    additional_infor_df._set_value(row,'list', str([list_temp[1], list_temp[0]]))
                else:
                    additional_infor_df._set_value(row,'list', str(list_temp))
            else:
                additional_infor_df._set_value(row,'list', str(list_temp))
        grouped = additional_infor_df.groupby('list')['count'].sum()
        additional_infor_df = pd.DataFrame({'list':[ast.literal_eval(index) for index in grouped.index]})
        additional_infor_df.insert(len(additional_infor_df.columns), 'count', list( grouped.iloc[:]))
        verify_list = [['02_09', '05_06'],['02_05', '06_09'],['02_06', '05_09']]
        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==2 and l in verify_list else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/df_all_entity.row_num)
    elif i in [10]:
        drop_index_list = []
        for j in range(len(additional_infor_df)):
            if additional_infor_df.iloc[j,0]== None:
                drop_index_list.append(j)
            elif len(additional_infor_df.iloc[j,0]) == 0:
                drop_index_list.append(j)
            elif type(additional_infor_df.iloc[j,0][0]) != str:
                drop_index_list.append(j)
        additional_infor_df = additional_infor_df.drop(drop_index_list)
        additional_infor_df = additional_infor_df.reset_index(drop=True)
        for row in range(len(additional_infor_df)):
            list_temp = additional_infor_df.loc[row, 'list']
        
            if list_temp!=None:
                list_temp = [[int(rebuild_i) for rebuild_i in re.findall(r"\d+", rebuild)] for rebuild in list_temp]
                for list_mem in list_temp:
                    list_mem.sort()
                list_temp.sort()
                additional_infor_df._set_value(row,'list', str([str(list_str[0]) + '_' + str(list_str[1]) for list_str in list_temp]))
            else:
                additional_infor_df._set_value(row,'list', str(list_temp))
        grouped = additional_infor_df.groupby('list')['count'].sum()
        additional_infor_df = pd.DataFrame({'list':[ast.literal_eval(index) for index in grouped.index]})        
        additional_infor_df.insert(len(additional_infor_df.columns), 'count', list( grouped.iloc[:]))
        additional_infor_df.insert(len(additional_infor_df.columns), 'success', ['1' if l!= None and len(l)==3 and len(set([re.findall(r"\d+",l[0])[0], re.findall(r"\d+",l[0])[1], re.findall(r"\d+",l[1])[0], re.findall(r"\d+",l[1])[1], re.findall(r"\d+",l[2])[0],re.findall(r"\d+",l[2])[1]]))==6 and re.findall(r"\d+",l[0])[0] in verify_list and re.findall(r"\d+",l[0])[1] in verify_list and re.findall(r"\d+",l[1])[0] in verify_list and re.findall(r"\d+",l[1])[1] in verify_list and re.findall(r"\d+",l[2])[0] in verify_list and re.findall(r"\d+",l[2])[1] in verify_list else '0' for l in additional_infor_df.iloc[:,0] ])
        accuracy_list.append(additional_infor_df.groupby('success')['count'].sum().iloc[1]/self.row_num)

    addition_list.append(additional_infor_df)
    additional_infor_df.to_excel('./output/all_acc'+'/' +str(i) + '_count.xlsx')

0
1
2
3
4
5
6
7
8
9
10


IndexError: single positional indexer is out-of-bounds

In [21]:
addition_list[5]


,list,count,success
0,"[A_B, A_C]",38,0
1,"[A_B, C_A]",472,0
2,"[A_B, C_B]",379,0
3,[A_B],116,0
4,"[A_C, B_A]",16,0
5,"[A_C, B_C]",28,0
6,[A_C],45,0
7,"[B_A, B_C]",13,0
8,"[B_A, C_A, G_F, D_B, E_B]",1,0
9,"[B_A, C_A]",26189,1


In [27]:
accuracy_list

[0.8438429882874328,
 0.703102247546692,
 0.8526432415321304,
 0.7790756568534346,
 0.8290281734726179,
 0.7917695473251029,
 0.8182969294080406,
 0.7529597974042419,
 0.79335232668566]

# 发现问题
1